In [1]:
import pandas as pd

holdings = pd.read_csv('../data/processed/holdings_clean.csv', parse_dates=['AsOfDate'])
trades = pd.read_csv('../data/processed/trades_clean.csv', parse_dates=['TradeDate', 'SettleDate'])

print("="*60)
print("KEY AGGREGATIONS FOR CHATBOT")
print("="*60)

KEY AGGREGATIONS FOR CHATBOT


Fund Level Aggregations

In [2]:
# 1. Current fund performance (most recent date)
latest_date = holdings['AsOfDate'].max()
current_holdings = holdings[holdings['AsOfDate'] == latest_date]

fund_summary = current_holdings.groupby('PortfolioName').agg({
    'SecurityId': 'count',  # Number of holdings
    'MV_Base': 'sum',       # Total market value
    'PL_YTD': 'sum',        # YTD P&L
    'PL_MTD': 'sum',        # MTD P&L
    'PL_QTD': 'sum',        # QTD P&L
}).rename(columns={'SecurityId': 'num_holdings'})

fund_summary = fund_summary.round(2)
print("\n1. FUND PERFORMANCE SUMMARY")
print(fund_summary.head(10))

# Add return calculations
fund_summary['ytd_return_pct'] = (fund_summary['PL_YTD'] / 
    (fund_summary['MV_Base'] - fund_summary['PL_YTD']) * 100).round(2)

# Best/worst performers
print("\nTop 5 performers (YTD):")
print(fund_summary.nlargest(5, 'PL_YTD')[['PL_YTD', 'ytd_return_pct']])

print("\nWorst 5 performers (YTD):")
print(fund_summary.nsmallest(5, 'PL_YTD')[['PL_YTD', 'ytd_return_pct']])


1. FUND PERFORMANCE SUMMARY
                     num_holdings       MV_Base        PL_YTD      PL_MTD  \
PortfolioName                                                               
CoYold 1                        7  0.000000e+00  2.105159e+04        0.00   
CoYold 11                       3  0.000000e+00 -1.120346e+08        0.00   
CoYold 7                        1  0.000000e+00 -5.000000e+09        0.00   
Garfield                      221  1.446802e+08 -1.685510e+08   -14170.14   
Heather                       195  1.318358e+08 -1.815971e+08   -50226.48   
Hi Yield                       19  2.428341e+06 -5.124479e+04    26014.79   
IG Corp                         1  0.000000e+00  2.070000e+00        0.00   
MNC Investment Fund           243  1.271527e+08 -3.286149e+08 -1410025.64   
NPSMF1                         17  1.949746e+07  3.043387e+05     -320.07   
NPSMF2                         17  1.974930e+07  2.975902e+05     -355.90   

                           PL_QTD  
PortfolioN

Trade Activity Aggregations

In [3]:
# 2. Trade volume by fund
trade_summary = trades.groupby('PortfolioName').agg({
    'id': 'count',           # Number of trades
    'TotalCash': ['sum', 'mean', 'median'],
    'Quantity': 'sum'
}).round(2)

trade_summary.columns = ['num_trades', 'total_cash', 'avg_trade_size', 'median_trade_size', 'total_quantity']

print("\n2. TRADE ACTIVITY BY FUND")
print(trade_summary.head(10))

# Most active funds
print("\nMost active funds (by trade count):")
print(trade_summary.nlargest(10, 'num_trades')['num_trades'])


2. TRADE ACTIVITY BY FUND
                  num_trades    total_cash  avg_trade_size  median_trade_size  \
PortfolioName                                                                   
Account A                  5  1.002549e+09    2.005098e+08       1.800000e+07   
Account B                  3  9.345491e+08    3.115164e+08       1.268000e+06   
Account C                  1  1.268000e+06    1.268000e+06       1.268000e+06   
Account D                  2  2.532000e+06    1.266000e+06       1.266000e+06   
CampNou Holdings          30  2.910346e+08    9.701152e+06       9.988673e+06   
ClientA                   24  2.895331e+08    1.206388e+07       4.422000e+05   
HoldCo 1                  43  2.747689e+08    6.389974e+06       4.201000e+05   
HoldCo 11                  6  1.794919e+08    2.991532e+07       9.999816e+06   
HoldCo 3                   1  2.553540e+09    2.553540e+09       2.553540e+09   
HoldCo 7                   1  5.000092e+09    5.000092e+09       5.000092e+09   



Time Based Aggregations

In [4]:
# 3. Monthly trade volume
trades['year_month'] = trades['TradeDate'].dt.to_period('M')

monthly_volume = trades.groupby('year_month').agg({
    'id': 'count',
    'TotalCash': 'sum'
}).rename(columns={'id': 'num_trades', 'TotalCash': 'total_volume'})

print("\n3. MONTHLY TRADE VOLUME")
print(monthly_volume.tail(12))

# 4. Daily activity (recent)
recent_trades = trades[trades['TradeDate'] >= trades['TradeDate'].max() - pd.Timedelta(days=30)]
daily_activity = recent_trades.groupby(recent_trades['TradeDate'].dt.date).size()

print("\n4. RECENT DAILY ACTIVITY")
print(daily_activity.tail(10))


3. MONTHLY TRADE VOLUME
            num_trades  total_volume
year_month                          
2026-01            649  1.369150e+10

4. RECENT DAILY ACTIVITY
TradeDate
2026-01-13    649
dtype: int64


Security Level Aggregations

In [5]:
# 5. Most held securities
security_holdings = current_holdings.groupby(['SecurityId', 'SecName']).agg({
    'PortfolioName': 'count',  # Number of funds holding
    'Qty': 'sum',
    'MV_Base': 'sum'
}).rename(columns={'PortfolioName': 'num_funds'})

print("\n5. MOST WIDELY HELD SECURITIES")
print(security_holdings.nlargest(10, 'num_funds'))

# 6. Most traded securities
most_traded = trades.groupby(['SecurityId', 'Name']).agg({
    'id': 'count',
    'Quantity': 'sum',
    'TotalCash': 'sum'
}).rename(columns={'id': 'num_trades'})

print("\n6. MOST TRADED SECURITIES")
print(most_traded.nlargest(10, 'num_trades'))


5. MOST WIDELY HELD SECURITIES
                      num_funds          Qty      MV_Base
SecurityId SecName                                       
288790     004421CG6        115  118350000.0         0.00
288800     177342AL6         40   93350000.0  61611000.00
289892     969457BM1         21   58000000.0         0.00
277144     F                 19   13306000.0  83162500.00
288857     FVZ6              16       5678.0  56780000.00
280313     WMT               15     180000.0  18793800.00
272584     SAP GR            14     597000.0  83482211.40
279455     GLD               14     414000.0  65912940.00
290030     BL1156233         14   63000000.0         0.00
273098     EJ0445951         13    7510100.0   9588895.68

6. MOST TRADED SECURITIES
                                      num_trades   Quantity     TotalCash
SecurityId Name                                                          
288790     004421CG6                          33  142640000  0.000000e+00
288800     177342AL6   

Strategy Direction Analysis

In [6]:
# 7. Strategy breakdown
if 'StrategyRefShortName' in holdings.columns:
    strategy_summary = current_holdings.groupby('StrategyRefShortName').agg({
        'MV_Base': 'sum',
        'PL_YTD': 'sum',
        'SecurityId': 'count'
    })
    print("\n7. STRATEGY BREAKDOWN")
    print(strategy_summary)

# 8. Long vs Short positions
direction_summary = current_holdings.groupby('DirectionName').agg({
    'MV_Base': 'sum',
    'PL_YTD': 'sum',
    'SecurityId': 'count'
})
print("\n8. LONG vs SHORT POSITIONS")
print(direction_summary)


7. STRATEGY BREAKDOWN
                           MV_Base        PL_YTD  SecurityId
StrategyRefShortName                                        
 Default              1.865967e+09 -6.053358e+09         521
AA                    5.880000e+06  3.599851e+05           8
ACCSS                 7.050000e+06  2.568493e+05           4
ACE                   0.000000e+00  0.000000e+00          31
AH4R                  7.176000e+06  1.577810e+05           4
...                            ...           ...         ...
UTX                   1.237598e+07  8.783000e+04          14
VELOCITY              1.650000e+03  1.500000e+02           4
VENEZUELA             1.162800e+07  5.700081e+05           4
WMB                   6.399000e+06  4.050031e+05           4
WMT                   1.576591e+07 -1.449600e+05           7

[64 rows x 3 columns]

8. LONG vs SHORT POSITIONS
                    MV_Base        PL_YTD  SecurityId
DirectionName                                        
Long           2.413841e+